# Integrations tests: NDT inside OaaS Platform v2

In [18]:
import requests
import json

URL = 'http://0.0.0.0:5000'
START_PORT = 55000

* Ejecutamos OaaS Node y comprobamos que podemos acceder a su API: <http://0.0.0.0:5000> 
* Partimos de que tenemos subido el NDT al registry de Optimaix
* El NDT esta subido al registry con el nombre: _elighthouse-ndt_

### 1. Dar del alta el Repository de NDT dentro de OaaS Node

In [19]:
# Comprobar que repositorios hay actualmente en la plataforma

# GET /algorithmRepository
response = requests.get(URL + '/images')

if response.status_code == 200:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    print("Failed to retrieve data")


{
    "items": [],
    "total": 0,
    "page": 1,
    "size": 50,
    "pages": 0
}


In [20]:
# Damos de alta el NDT como repositorio de tipo general (no algoritmos inside)

req = {
    "name": "NDT Repository",
    "description": "Network Digital Twin based in Net2Plan",
    "imageUrl": "elighthouse-ndt",
    "type": "java"
}

print(req)

# Sent a POST to /algorithmRepository
response = requests.post(URL + '/images', json=req)

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")


{'name': 'NDT Repository', 'description': 'Network Digital Twin based in Net2Plan', 'imageUrl': 'elighthouse-ndt', 'type': 'java'}
{
    "imageId": 1,
    "name": "NDT Repository",
    "description": "Network Digital Twin based in Net2Plan",
    "imageUrl": "elighthouse-ndt",
    "type": "java",
    "instances": [],
    "resources": [
        {
            "resourceId": 1,
            "localId": 0,
            "name": "elighthouse-ndt",
            "version": "1.0",
            "description": "Direct Algorithm elighthouse-ndt",
            "execution_type": "direct",
            "inputParametersTemplate": {},
            "outputParametersTemplateSuccess": {},
            "outputParametersTemplateFailure": null
        }
    ]
}


### 2. Instanciar NDT dentro de plataforma OaaS

In [21]:
# Comprobar que repositorios hay actualmente en la plataforma, deberia existir el repositorio NDT

# GET /algorithmRepository
response = requests.get(URL + '/images')

if response.status_code == 200:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    print("Failed to retrieve data")

{
    "items": [
        {
            "imageId": 1,
            "name": "NDT Repository",
            "description": "Network Digital Twin based in Net2Plan",
            "imageUrl": "elighthouse-ndt",
            "type": "java",
            "instances": [],
            "resources": [
                {
                    "resourceId": 1,
                    "localId": 0,
                    "name": "elighthouse-ndt",
                    "version": "1.0",
                    "description": "Direct Algorithm elighthouse-ndt",
                    "execution_type": "direct",
                    "inputParametersTemplate": {},
                    "outputParametersTemplateSuccess": {},
                    "outputParametersTemplateFailure": null
                }
            ]
        }
    ],
    "total": 1,
    "page": 1,
    "size": 50,
    "pages": 1
}


In [22]:
image_ndt_id = 1

In [24]:
# Creamos una nueva instancia del NDT

req = {
    "name": "NDT Repository Instance 1",
    "description": "Network Digital Twin based in Net2Plan",
    "quotaInformation": {
        "cpu": "1 core",
        "diskSpace": "10GB",
        "networkBandwidth": "100Mbps",
        "ram": "512MB"
    },
    "image_id": image_ndt_id
}

print(req)

# Sent a POST to /algorithmRepository
response = requests.post(URL + '/instances', json=req)

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{'name': 'NDT Repository Instance 1', 'description': 'Network Digital Twin based in Net2Plan', 'quotaInformation': {'cpu': '1 core', 'diskSpace': '10GB', 'networkBandwidth': '100Mbps', 'ram': '512MB'}, 'image_id': 1}
{
    "instanceId": 1,
    "name": "NDT Repository Instance 1",
    "description": "Network Digital Twin based in Net2Plan",
    "status": "running",
    "quotaInformation": {
        "cpu": "1 core",
        "diskSpace": "10GB",
        "networkBandwidth": "100Mbps",
        "ram": "512MB"
    },
    "image": {
        "imageId": 1,
        "name": "NDT Repository",
        "description": "Network Digital Twin based in Net2Plan",
        "imageUrl": "elighthouse-ndt"
    }
}


In [25]:
instance_id = 1

In [26]:
# Start instance of NDT
response = requests.post(URL + f'/instances/{instance_id}/start')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{
    "detail": "Instance 1 is already started"
}
Failed to retrieve data


### 3. Access to NDT directly

In [27]:
NDT_URL = f"http://localhost:{START_PORT + instance_id}"
print(NDT_URL)

http://localhost:55001


In [29]:
# Get Current Topology directly from the NDT instance inside OaaS Plataform
endpoint = "/current-topology"
response = requests.get(URL + f'/instances/{instance_id}/execute' + f'{endpoint}')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
elif response.status_code == 422:
    print(response.content)
    print("No topology available")
else:
    print("Failed to retrieve data")

b'{"detail":"Error executing GET request on instance 1. "}'
No topology available


### 4. OaaS Platform to interactuate with NDT

In [28]:
# Stop Instance of NDT
response = requests.delete(URL + f'/instances/{instance_id}/stop')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{
    "message": "Stopped instance 1"
}


In [75]:
# Start instance of NDT
response = requests.post(URL + f'/instances/{instance_id}/start')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{
    "message": "Started instance 2"
}


In [76]:
# Get Instance of NDT
response = requests.get(URL + f'/instances/{instance_id}')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{
    "id": 2,
    "name": "NDT Repository Instance 1",
    "description": "Network Digital Twin based in Net2Plan",
    "status": "running",
    "quotaInformation": "none",
    "repository": {
        "id": 20,
        "name": "NDT Repository",
        "description": "Network Digital Twin based in Net2Plan",
        "url": "elighthouse-ndt",
        "type": "java"
    }
}


In [15]:
# Delete instance of NDT
response = requests.delete(URL + f'/instances/{instance_id}/delete')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{
    "detail": "Instance 1 is already started"
}
Failed to retrieve data
